<a href="https://colab.research.google.com/github/sarvatgit/2018201009_lab4/blob/master/Category_ranking_using_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')
# data_folder = '/content/drive/My Drive/pg2k18/sem2/smai/smai_proj/reuters21578/'
data_folder = '/content/drive/My Drive/reuters21578/'

sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
#     print(i[1])
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)

In [40]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X, document_Y):
    with open(filename, 'rb') as file:
        content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
        document_categories = []
        document_id = newsline['newid']
        document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
        if document_body == 'None':
            continue
        doc_categories = strip_tags(str(newsline('topics')[0].body))
        doc_categories = unescape(doc_categories)

        document_body = unescape(document_body)
        topics = newsline.topics.contents
        places = newsline.places.contents
        people = newsline.people.contents
        orgs = newsline.orgs.contents
        exchanges = newsline.exchanges.contents
        for topic in topics:
            document_categories.append('to_' + strip_tags(str(topic)))

        for place in places:
            document_categories.append('pl_' + strip_tags(str(place)))
        for person in people:
            document_categories.append('pe_' + strip_tags(str(person)))
        for org in orgs:
            document_categories.append('or_' + strip_tags(str(org)))
        for exchange in exchanges:
            document_categories.append('ex_' + strip_tags(str(exchange)))
        document_X[document_id] = document_body
        document_Y[document_id] = document_categories

def readFiles(test_data = False):
    document_X = {}
    document_Y = {}
    if test_data == True:
        file_name = sgml_file_name_template.replace('NNN', '021')
        filename = data_folder + file_name
        makeDict(filename, document_X, document_Y)
    else:
        for i in range(sgml_number_of_files):
            if i < 10:
                seq = '00' + str(i)
            else:
                seq = '0' + str(i)
        file_name = sgml_file_name_template.replace('NNN', seq)
        print('Reading file: %s' % file_name)
        filename = data_folder + file_name
        makeDict(filename, document_X, document_Y)
    return document_X, document_Y



In [43]:
document_X, document_Y = readFiles()

Reading file: reut2-020.sgm


In [44]:
nltk.download('wordnet')
def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:


words_in_body={}

for i in range(len(totalX)):
    words=totalX[i].split(' ')
    words_in_body[i]=words    

one_hot_label=[]
for key,v in words_in_body.items():
    dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
    for i in v:
        if i in category_dictionary['Topics']:
            dict_temp['Topics']+=1
        if i in category_dictionary['Places']:
            dict_temp['Places']+=1
        if i in category_dictionary['People']:
            dict_temp['People']+=1
        if i in category_dictionary['Exchanges']:
            dict_temp['Exchanges']+=1
        if i in category_dictionary['Organizations']:
            dict_temp['Organizations']+=1
            
    one_hot_label.append(dict_temp)
    




In [47]:
#print(one_hot_label)
import operator
true_ranks=[]
for i in one_hot_label:
    true_ranks.append(sorted(i.items(),key=operator.itemgetter(1),reverse=True))
print(true_ranks[0])
#print(totalX[0])

[('Topics', 7), ('People', 3), ('Places', 1), ('Exchanges', 0), ('Organizations', 0)]


In [0]:
# data preprocessing
import numpy as np

#nltk.download('punkt')
#nltk.download('wordnet')

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [0]:
vectorizer_svm = TfidfVectorizer(stop_words='english')
mlb = MultiLabelBinarizer()
svmlabeltrain=[]
svmlabeltest=[]

In [67]:
for i in true_ranks[0:500]:
    svm=[]
    for j in i:
        if j[1]!=0:
            svm.append(j[0])
    svmlabeltrain.append(svm)
print(svmlabeltrain[0])

['Topics', 'People', 'Places']


In [0]:
for i in true_ranks[500:700]:
    svm=[]
    for j in i:
        if j[1]!=0:
            svm.append(j[0])
    svmlabeltest.append(svm)


In [0]:
mlb = MultiLabelBinarizer()
#print("svmlabeltrain: ",type(svmlabeltrain),len(svmlabeltrain))
Y = mlb.fit_transform(svmlabeltrain)

Y=np.array(Y)
X=np.array(totalX[0:500])

In [85]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])
#print(Y[0])
classifier.fit(X, Y)
print("X:",X.shape)
X_test=np.array(totalX[500:700])
print(totalX[600])

predicted = classifier.predict(X_test)
# print("X_test:",X_test.shape)
print("predicted: ",predicted.shape, type(predicted))
#print(predicted)
print(true_ranks[0])
#print(totalX[500])

X: (500,)
shr 83 cts vs 67 cts net 10 052 000 vs 7 929 000 avg shrs 12 161 000 vs 11 824 000 nine mths shr 233 dlrs vs 191 dlrs net 28 110 000 vs 22 386 000 avg shrs 12 078 000 vs 11 729 000 assets 59 billion vs 57 billion deposits 53 billion vs 48 billion loans leases 42 billion vs 40 billion reuter
predicted:  (200, 5) <class 'numpy.ndarray'>
[('Topics', 7), ('People', 3), ('Places', 1), ('Exchanges', 0), ('Organizations', 0)]


In [90]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import LogisticRegression, RidgeClassifier
# tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
# X_train = tfidf_vectorizer.fit_transform(totalX[0:500])
#    # X_val = tfidf_vectorizer.transform(X_val)
# X_test = tfidf_vectorizer.transform(totalX[500:700])
# y_train = mlb.fit_transform(svmlabeltrain)
# clf = LogisticRegression(C=5,solver='liblinear')
#     #clf = OneVsRestClassifier(clf)
# clf.fit(X_train, y_train)

ValueError: ignored

In [55]:
# print(predicted)
all_labels = mlb.inverse_transform(predicted)
print(all_labels[0])
# print(len(svmlabeltest))

()
